<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [80]:
# 1. load the data

import pandas as pd

# mount dataframe
telcom = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [81]:
telcom.shape

(7043, 21)

In [82]:
telcom.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [0]:
# convert target variable to binary
telcom['Churn'] = telcom['Churn'].replace({'Yes': 1, 'No': 0})

In [0]:
# drop customerID as cannot be encoded
X = telcom.drop(columns=['Churn', 'customerID'])
y = telcom['Churn']

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5282, 19), (5282,), (1761, 19), (1761,))

In [86]:
!pip install category_encoders

In [0]:
# 2. clean data

# encode 
import category_encoders as ce
encoder = ce.OneHotEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

# normalize
from sklearn.preprocessing import MinMaxScaler, Normalizer

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [0]:
# X Variable Types
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
# y needs to be transformed into an array???
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()

In [0]:
# 3. baseline 

# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 50
batch_size = 10

In [92]:
# create model without tuning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(15, activation='relu', input_shape=(inputs,)))
model.add(Dense(10, activation='relu'))

# add sigmoid for output layer,  because classification (see notes)
model.add(Dense(1, activation='sigmoid'))
          
  
# since dependent variable is binary, use the log loss function 
# if there are > 2 categroies in output, then use categorical_crossentropy

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size,
          verbose=True
         )

Train on 5282 samples, validate on 1761 samples
Epoch 1/50
5282/5282 [==============================] - 2s 429us/sample - loss: 0.4619 - acc: 0.7673 - val_loss: 0.4338 - val_acc: 0.7785
Epoch 2/50
5282/5282 [==============================] - 2s 393us/sample - loss: 0.4079 - acc: 0.8116 - val_loss: 0.4284 - val_acc: 0.7888
Epoch 3/50
5282/5282 [==============================] - 2s 387us/sample - loss: 0.3651 - acc: 0.8431 - val_loss: 0.4161 - val_acc: 0.8018
Epoch 4/50
5282/5282 [==============================] - 2s 396us/sample - loss: 0.2838 - acc: 0.8944 - val_loss: 0.4367 - val_acc: 0.7791
Epoch 5/50
5282/5282 [==============================] - 2s 393us/sample - loss: 0.1605 - acc: 0.9580 - val_loss: 0.4630 - val_acc: 0.7774
Epoch 6/50
5282/5282 [==============================] - 2s 394us/sample - loss: 0.0797 - acc: 0.9760 - val_loss: 0.5462 - val_acc: 0.7286
Epoch 7/50
5282/5282 [==============================] - 2s 376us/sample - loss: 0.0544 - acc: 0.9786 - val_loss: 0.5577 - va

- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters

In [0]:
# 4. hyperparameter tuning

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


def create_model():
    # create model
    model = Sequential()
    model.add(Dense(15, activation='relu', input_shape=(inputs,)))
    model.add(Dense(10, activation='relu'))
    # add sigmoid for output layer,  because classification (see notes)
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model
  
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [0]:
# define the grid search parameters
param_grid = {'batch_size': [10],
              'epochs': [20, 40, 60, 200],
              'optimizer': ['adam', 'sgd'] ,
              'learning_rate': [0.001, 0.01] ,
              'momentum': [0, 0.2] ,
              'activation': ['softmax', 'relu', 'linear'],
              'init_mode': ['uniform', 'normal', 'zero'],
              'dropout_rate': [0.0, 0.1, 0.2],
              'neurons': [1, 5, 10]
              }

In [0]:
from sklearn.model_selection import GridSearchCV

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train, epochs=50)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?